In [193]:
#Load Libraries/modules
from Bio import SeqUtils
from Bio import SeqIO
import pandas as pd
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
from random import randint
%matplotlib inline

In [194]:
#Set standard elements of the gRNA oligo into items
BbsI = 'GAAGACggTATT'
Scaffold = 'GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGC'
AvrII = 'CCTAGG'
PstI = 'CTGCAG'

In [195]:
#Reverse Complement Function 
from Bio.SeqRecord import SeqRecord

def make_rc_record(record):
    """Returns a new SeqRecord with the reverse complement sequence."""
    return SeqRecord(seq = record.seq.reverse_complement(), \
                 id = record.id, \
                 description = "reverse complement")


In [196]:
#Reverse Complement HR1 and HR2

HR1_fasta = "/Users/srchernandez/Desktop/CRISPR_PbHIT_KO_Vector/PbHIT_KO_Vector_Final/corrected_GENE_SEQ_WITH_100_FLANKING_BASES_HR1.fasta"
records = map(make_rc_record, SeqIO.parse(HR1_fasta, "fasta"))
SeqIO.write(records, "HR1_rev_comp.fasta", "fasta")

HR2_fasta = "/Users/srchernandez/Desktop/CRISPR_PbHIT_KO_Vector/PbHIT_KO_Vector_Final/corrected_GENE_SEQ_WITH_100_FLANKING_BASES_HR2.fasta"
records = map(make_rc_record, SeqIO.parse(HR2_fasta, "fasta"))
SeqIO.write(records, "HR2_rev_comp.fasta", "fasta")


4945

In [183]:
#Read gRNA excel files as table

#this file has top 3 gRNAs for each gene, read only until column C 'Total_score'
gRNA_EuPaGDT_top = pd.read_excel("/Users/srchernandez/Desktop/CRISPR_PbHIT_KO_Vector/PbHIT_KO_Vector_Final/selected_gRNA.PbHIT_KO_Test.xlsx",index_col=None, na_values=['NA'], usecols="A:C")
gRNA_EuPaGDT_top[['GENE ID', 'gRNA ID','directionality']] = gRNA_EuPaGDT_top.gRNA_id.str.split("_", expand = True)
#gRNA_EuPaGDT_top.rename(columns={'gRNA_id':'GENE ID'}, inplace = True)
gRNA_EuPaGDT_top['GENE ID']=gRNA_EuPaGDT_top['GENE ID'].replace('PBANKA','PBANKA_', regex= True)

gRNA_EuPaGDT_top.head(5)

#this file has all gRNAs for each gene, read only until column C 'Total_score'
#gRNA_EuPaGDT_all = pd.read_excel("/Users/srchernandez/Desktop/CRISPR_PbHIT_KO_Vector/PbHIT_KO_Vector_Final/all_gRNA.PbHIT_KO_Test.xlsx",index_col=None, na_values=['NA'], usecols="A:C")
#gRNA_EuPaGDT_all[['GENE ID', 'gRNA ID','directionality']] = gRNA_EuPaGDT_all.gRNA_id.str.split("_", expand = True)
#gRNA_EuPaGDT_all.rename(columns={'gRNA_id':'GENE ID'}, inplace = True)
#gRNA_EuPaGDT_all['GENE ID']=gRNA_EuPaGDT_all['GENE ID'].replace('PBANKA','PBANKA_', regex= True)

#gRNA_EuPaGDT_all.head(5)

,gRNA_id,gRNA_sequence,Total_score,GENE ID,gRNA ID,directionality
0,PBANKA1119700_1246,TATGGCTCTGCTTCTGTTGA,0.82,PBANKA_1119700,1246,None
1,PBANKA1119700_1078,TCAATGTATGTACACCCATG,0.81,PBANKA_1119700,1078,None
2,PBANKA1119700_807_revcom,TATGGTACTATCCGAGCTAG,0.76,PBANKA_1119700,807,revcom
3,PBANKA0807400_441_revcom,TATGCCCGCTTTATGAGATG,0.78,PBANKA_0807400,441,revcom
4,PBANKA0807400_290_revcom,TCCCTTTCTACTCTTATAAC,0.72,PBANKA_0807400,290,revcom


In [184]:
#Create a DataFrame that has the Gene ID, HR1, and HR2
#Read HR1 FASTA file 
HR1_fasta_rev = "/Users/srchernandez/Desktop/CRISPR_PbHIT_KO_Vector/PbHIT_KO_Vector_Final/HR1_rev_comp.fasta"
HR1_seq_rev= [i for i in SeqIO.parse(HR1_fasta_rev,'fasta')]

#Store HR1 sequences into a string
genes = []
HR1_seq_rev = []
for seq_record in SeqIO.parse(HR1_fasta_rev,'fasta'):
    genes.append(seq_record.id)
    HR1_seq_rev.append(str(seq_record.seq))

#to see items in a FASTA file 
#PBANKA1 = HR1_seq[0]
#print (PBANKA1)

#Read HR2 FASTA file 
HR2_fasta_rev = "/Users/srchernandez/Desktop/CRISPR_PbHIT_KO_Vector/PbHIT_KO_Vector_Final/HR2_rev_comp.fasta"
HR2_seq_rev= [i for i in SeqIO.parse(HR2_fasta_rev,'fasta')]

#Store HR2 sequences into a string 
genes = []
HR2_seq_rev = []

for seq_record in SeqIO.parse(HR2_fasta_rev,'fasta'):
    genes.append(seq_record.id)
    HR2_seq_rev.append(str(seq_record.seq))

#Generate table with Genes, HR1, HR2

pHIT_KO_HR = pd.DataFrame({
    "GENE ID": genes,
    "HR1 Sequence": HR1_seq_rev,
    "HR2 Sequence": HR2_seq_rev
})

#pHIT_KO_HR.head(5)

In [189]:
#Load Gene List
genes = pd.read_excel("/Users/srchernandez/Desktop/CRISPR_PbHIT_KO_Vector/PbHIT_KO_Vector_Final/PbHiT_all_genes_E.xlsx",na_values=['NA'])
gene_list=genes['Target Genes'].tolist()
genes.head(5)

,Target Genes
0,PBANKA_1214700
1,PBANKA_0514700
2,PBANKA_1308200
3,PBANKA_0105000
4,PBANKA_1114600


In [190]:
#Convert to batch search
#gene_list=['PBANKA_1006200']
#rows=len(gene_list)
#dftest=pd.DataFrame()

for x in gene_list:
    input_gene=x
    gene_gRNA=gRNA_EuPaGDT_top[gRNA_EuPaGDT_top['GENE ID']==input_gene]
    if not gene_gRNA.empty:
        Result1 = BbsI + gene_gRNA['gRNA_sequence']
        #print(Result1)
    else:
        print('No gRNA')
    
    #Store Results in a data frame 
    pHIT_KO_BbsI_gRNA = pd.DataFrame({
        "GENE ID": input_gene,
        "Sequence": Result1
    })
    
    pHIT_KO_BbsI_gRNA_top2=pHIT_KO_BbsI_gRNA.iloc[:3]
    
    gene_HR=pHIT_KO_HR[pHIT_KO_HR['GENE ID']==input_gene]
    if not gene_HR.empty:
        Result2= Scaffold + gene_HR['HR2 Sequence'] + AvrII + gene_HR['HR1 Sequence']+ PstI
        #print(Result2)
    else: 
        print('Gene Cannot be Found')

        #Convert Result2 into a list
    pHIT_KO_HR_seq=Result2.values.tolist()

        #Generate final oligo
    PbHOT_KO_Vector_List=pd.DataFrame(columns=['GENE ID', 'Oligo Sequence'])
    PbHOT_KO_Vector_List['GENE ID']=pHIT_KO_BbsI_gRNA_top2['GENE ID']
    PbHOT_KO_Vector_List['Oligo Sequence']=pHIT_KO_BbsI_gRNA_top2['Sequence']+pHIT_KO_HR_seq
    
    new_row=PbHOT_KO_Vector_List
    dftest=pd.concat([dftest,new_row])
    #df=df.append(new_row,ignore_index=True)
    
#pHIT_KO_BbsI_gRNA.head(10)

#dftest.head(10)
    

No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene Cannot be Found
No gRNA
Gene C

In [191]:
dftest.to_csv("/Users/srchernandez/Desktop/PbHiT_KO_Vector_all_genes_E", index=None)

In [192]:
dftest.to_excel("/Users/srchernandez/Desktop/PbHiT_KO_Vector_all_genes_E.xlsx", index=None)